# Exploración Avanzada del Dataset Clínico

In [ ]:
# Librerías necesarias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Carga del dataset
df = pd.read_csv('/mnt/data/data.csv')
df.head()

## 1. PCA - Reducción de Dimensionalidad

In [ ]:
numeric_data = df.select_dtypes(include=[np.number]).dropna()
scaled = StandardScaler().fit_transform(numeric_data)
pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled)

plt.figure(figsize=(8,6))
sns.scatterplot(x=pca_result[:,0], y=pca_result[:,1], hue=df.loc[numeric_data.index, 'tipo_cancer_general'])
plt.title("PCA - Reducción de Dimensionalidad")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.show()

## 2. Agrupación Jerárquica de Variables Categóricas

In [ ]:
pivot = pd.crosstab(df['tipo_cancer_general'], df['gender.demographic'])
sns.clustermap(pivot, cmap="viridis", standard_scale=1)

## 3. Análisis Multivariado con Pairplot

In [ ]:
numeric_cols = df.select_dtypes(include=np.number).columns[:5]
sns.pairplot(df, hue='tipo_cancer_general', vars=numeric_cols)

## 4. Outliers por Clase de Cáncer

In [ ]:
for col in df.select_dtypes(include=[np.number]).columns:
    plt.figure(figsize=(10, 4))
    sns.boxplot(data=df, x='tipo_cancer_general', y=col)
    plt.title(f'Distribución de {col} por tipo de cáncer')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 5. Perfilado Automático (opcional con ydata-profiling)

In [ ]:
# from ydata_profiling import ProfileReport
# profile = ProfileReport(df, title='Reporte Exploratorio', explorative=True)
# profile.to_notebook_iframe()

## 6. Patrones de Valores Faltantes

In [ ]:
msno.dendrogram(df)

## 7. Detección de Duplicados e Inconsistencias

In [ ]:
duplicated = df[df.duplicated(subset='sample_id.samples', keep=False)]
duplicated[['sample_id.samples', 'case_id', 'tipo_cancer_general']]